# Mean Depth Estimation Error (mDEE) for Monocular Depth Estimation (MDE)

$$
\text{mDEE} = \frac{1}{N \times S} \sum_{i=1}^{N} \sum_{s=1}^{S} \left(\frac{\text{E}_{i,s}}{\text{E}_{\text{base},i,s}} \right)
$$

To incorporate different perturbations and severities into the mDEE equation, we adjust it to account for the varying impact of each type of corruption on depth estimation accuracy. This extended formula integrates the severity levels of perturbations:



$N$ is the number of perturbations, $S$ is the number of severity levels for each perturbation

$\text{E}_{i,s}$ is the error for the $i_{th}$ perturbation at severity level $s$ 

$\text{E}_{\text{base},i,s}$ is the baseline error for the same

$\alpha_{i,s}$ and $\beta_{i,s}$ are weights for accuracy and consistency at each perturbation and severity level

$\text{Var}_{i,s}$ is the variance in depth estimation error for added robustness assessment.

In [20]:
import torch

# Extended mDEE calculation including perturbation and severity
def calculate_extended_mdee(predictions, ground_truth, baseline_errors, alpha, beta):

    # Assuming BxCxHxW format
    depth_errors = torch.abs(predictions - ground_truth).mean(dim=[1, 2, 3])  

    normalized_errors = depth_errors / baseline_errors

    # Alpha and Beta errors and variances
    weighted_errors = alpha * normalized_errors
    weighted_variances = beta * torch.var(normalized_errors, unbiased=False)

    mdee = (weighted_errors + weighted_variances).mean()
    return mdee



# Example

In [21]:
import torch
import torch.nn as nn
import numpy as np

class SimpleDepthModel(nn.Module):
    def __init__(self):
        super(SimpleDepthModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(16, 1, kernel_size=3, stride=1, padding=1)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.conv2(x)
        return x


In [22]:
def naive_calculate_mdee(predictions, ground_truth):
    depth_errors = torch.abs(predictions - ground_truth)
    mean_depth_errors = depth_errors.mean()
    return mean_depth_errors

# Example usage
model = SimpleDepthModel()
input_tensor = torch.rand(10, 3, 224, 224)
ground_truth = torch.rand(10, 1, 224, 224)
predictions = model(input_tensor)

In [23]:
# Calculate mDEE
mdee_score = naive_calculate_mdee(predictions, ground_truth)
print(f"Mean Depth Estimation Error (mDEE): {mdee_score.item()}")

Mean Depth Estimation Error (mDEE): 0.5277677774429321


## Example perturbation

In [24]:
def add_gaussian_noise(images, severity):
    noise = torch.randn_like(images) * severity
    return images + noise

## Initialize model, inputs, and ground truth

In [25]:
model = SimpleDepthModel()
input_tensor = torch.rand(10, 3, 224, 224)  # Batch of images
ground_truth = torch.rand(10, 1, 224, 224)  # Corresponding ground truth depth maps

### Severity Levels for Perturbation (Gaussian)

In [26]:
severity_levels = [0.01, 0.05, 0.1]

alpha = 1  # Weight for accuracy
beta = 0.1  # Weight for consistency

# TODO: baseline errors from another model
# Example baseline error for normalization
baseline_errors = torch.tensor([0.05])  

### Evaluate mDEE under different noise severities

In [33]:
history_mdee_score = []

for severity in severity_levels:
    noisy_input = add_gaussian_noise(input_tensor, severity)
    predictions = model(noisy_input)
    partial_mdee_score = calculate_extended_mdee(predictions, ground_truth, baseline_errors, alpha, beta)
    history_mdee_score.append(partial_mdee_score.item())
    print(f"Partial mDEE with noise severity {severity}: {partial_mdee_score.item()}")

Partial mDEE with noise severity 0.01: 10.845612525939941
Partial mDEE with noise severity 0.05: 10.854713439941406
Partial mDEE with noise severity 0.1: 10.882315635681152


In [34]:
history_mdee_score

[10.845612525939941, 10.854713439941406, 10.882315635681152]

In [35]:
np.mean(history_mdee_score)

10.860880533854166

# More Perturbations

In [41]:
import torch
import torch.nn as nn
import torch.nn.functional as F

def adjust_brightness(images, brightness_factor):
    return images * brightness_factor

def adjust_contrast(images, contrast_factor):
    means = torch.mean(images, dim=[2, 3], keepdim=True)
    return (images - means) * contrast_factor + means

def apply_blur(images, kernel_size):
    padding = kernel_size // 2
    for i in range(images.shape[0]):
        images[i] = F.conv2d(images[i].unsqueeze(0), weight=get_gaussian_kernel(kernel_size), padding=padding, groups=3)
    return images

def get_gaussian_kernel(kernel_size, sigma=2):
    x = torch.arange(kernel_size).float() - kernel_size // 2
    gauss = torch.exp(-x**2 / float(2 * sigma**2))
    gauss = gauss / gauss.sum()
    _1D_kernel = gauss.unsqueeze(1)
    _2D_kernel = _1D_kernel.mm(_1D_kernel.t())
    kernel = _2D_kernel.expand(3, 1, kernel_size, kernel_size)
    return kernel

In [45]:
# Perturbations and Severities
perturbations = [
    (add_gaussian_noise, [0.01, 0.05, 0.1]),
    (adjust_brightness, [0.8, 1, 1.2]),
    (adjust_contrast, [0.8, 1, 1.2]),
    (apply_blur, [3, 5, 7])
]

In [46]:
# Example implementation for all perturbations type and severity
for perturb_func, severities in perturbations:
    for severity in severities:
        perturbed_input = perturb_func(input_tensor, severity)
        predictions = model(perturbed_input)

        
        # Calculate and print mDEE for each perturbation and severity
        mdee_score = calculate_extended_mdee(predictions, ground_truth, baseline_errors, alpha, beta)
        print(f"Extended mDEE with {perturb_func.__name__} severity {severity}: {mdee_score.item()}")

Extended mDEE with add_gaussian_noise severity 0.01: 10.438209533691406
Extended mDEE with add_gaussian_noise severity 0.05: 10.451985359191895
Extended mDEE with add_gaussian_noise severity 0.1: 10.491411209106445
Extended mDEE with adjust_brightness severity 0.8: 9.809255599975586
Extended mDEE with adjust_brightness severity 1: 10.437601089477539
Extended mDEE with adjust_brightness severity 1.2: 11.12680435180664
Extended mDEE with adjust_contrast severity 0.8: 10.427228927612305
Extended mDEE with adjust_contrast severity 1: 10.437601089477539
Extended mDEE with adjust_contrast severity 1.2: 10.449738502502441
Extended mDEE with apply_blur severity 3: 10.43205451965332
Extended mDEE with apply_blur severity 5: 10.419679641723633
Extended mDEE with apply_blur severity 7: 10.404075622558594


# Example: Baseline vs Our Model

In [47]:
class BaselineDepthModel(nn.Module):
    def __init__(self):
        super(BaselineDepthModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 8, kernel_size=5, stride=1, padding=2)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(8, 1, kernel_size=5, stride=1, padding=2)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.conv2(x)
        return x

In [48]:
primary_model = SimpleDepthModel()
baseline_model = BaselineDepthModel()

# Example input and ground truth
input_tensor = torch.rand(10, 3, 224, 224)
ground_truth = torch.rand(10, 1, 224, 224)

In [67]:
def _mdee(model, input_tensor, baseline_errors = 1):
    perturbation_list = []

    for perturb_func, severities in perturbations:
        
        severity_list = []
        for severity in severities:
            perturbed_input = perturb_func(input_tensor, severity)
            predictions = model(perturbed_input)

            
            # Calculate and print mDEE for each perturbation and severity
            mdee_score = calculate_extended_mdee(predictions, ground_truth, baseline_errors, alpha, beta)
            severity_list.append(mdee_score.item())

            # print(f"Extended mDEE with {perturb_func.__name__} severity {severity}: {mdee_score.item()}")
        
        perturbation_list.append(np.mean(severity_list))
    
    return np.mean(perturbation_list)

### Baseline Model

In [68]:
with torch.no_grad():
    base_performance = _mdee(baseline_model, input_tensor)
    print("mDEE: ", base_performance)

mDEE:  0.6669695923725764


### Our Model

In [69]:
with torch.no_grad():
    our_performance = _mdee(primary_model, input_tensor, base_performance)
    print("mDEE: ", our_performance)

mDEE:  0.7470979541540146


base: https://github.com/nianticlabs/monodepth2 (en espera de entrenado)

model1: https://github.com/ShuweiShao/AF-SfMLearner (ya todo en SCARED) en este empiezo

model2: https://github.com/CapsuleEndoscope/EndoSLAM/tree/master/EndoSfMLearner (en espera de model entrenado)

model3: https://github.com/zxcqlf/MonoViT/tree/main (en espera de entrenado)

our: https://github.com/RicardoEspinosaLoera/endo-manydepth/tree/master/manydepth (pendiente de modelo)
